In [4]:
import glob
import re
import gc

import numpy as np
import pandas as pd
import polars as pl

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## 0. Load Training Data

In [5]:
# file_paths = glob.glob("./input/train.parquet/partition_id=*/part-*.parquet")
# file_paths.sort()
# lazy_dfs = []

# for idx, file_path in enumerate(tqdm(file_paths)):
#     lazy_df = pl.read_parquet(file_path).lazy().with_columns(
#         pl.lit(idx).alias("partition_id")
#     )
    
#     lazy_dfs.append(lazy_df)


# del lazy_df
# gc.collect()
# train_df = pl.concat(lazy_dfs)

# del lazy_dfs
# gc.collect()

The entire dataset has around 47M rows

In [ ]:
train_df = pl.scan_parquet("./preprocessed_dataset/training.parquet")
# train_df = train_df.sort(["date_id", "time_id", "symbol_id"])
train_df.collect()

### Check the number of unique date_ids

There are 1699 unique `date_id`s in training data

In [ ]:
# check number of unique date_id
train_df.select(pl.col('date_id').unique()).collect()

### Check the number of timesteps in a given day (across all symbols)

In [ ]:
res = train_df.group_by("date_id").agg(
    pl.col("time_id").count().alias("num_timesteps")
).sort(by=['date_id']).collect()

res


In [ ]:
res.to_pandas().plot(x="date_id", y="num_timesteps")
plt.title('Number of Timesteps by Date ID')
plt.xlabel('Date ID')
plt.ylabel('Number of Timesteps')
plt.tight_layout()
plt.show()

### Check the number of unique timesteps in a given day (across all symbols)

In [ ]:
res = train_df.group_by("date_id").agg(
    pl.col("time_id").n_unique().alias("num_unique_timesteps")
).sort(by=['date_id']).collect()

res

In [ ]:
# make the above table into a line chart
res.to_pandas().plot(x="date_id", y="num_unique_timesteps")
plt.show()

In [ ]:
res.group_by("num_unique_timesteps").agg(
    pl.col("date_id").n_unique().alias("num_occurrences")
)

First half of the data (677 days) has 849 unique timesteps per day, and the second half (1022 days) has 968 unique timesteps per day.

### Check the number of unique symbols

In [ ]:
# check number of unique symbol_ids
train_df.select(pl.col('symbol_id').unique()).collect()

### Check number of symbols by `date_id`

In [ ]:
# number of symbol_ids per date_id
res = train_df.group_by("date_id").agg(
    pl.col("symbol_id").n_unique().alias("num_unique_symbols")
).sort(by=['date_id']).collect()


res

In [ ]:
res.to_pandas().plot(x="date_id", y="num_unique_symbols")
plt.title('Number of Unique Symbols by Date ID')
plt.xlabel('Date ID')
plt.ylabel('Number of Unique Symbols')
plt.tight_layout()
plt.show()

In [ ]:
# check number of time_ids for each symbol_id
res = train_df.group_by("symbol_id").agg(
    pl.col("time_id").count().alias("num_timesteps")
).sort(by=['symbol_id']).collect()

res

In [ ]:
res.to_pandas().plot(x="symbol_id", y="num_timesteps", kind='bar')
plt.title('Number of Timesteps by Symbol ID')
plt.xlabel('Symbol ID')
plt.ylabel('Number of Timesteps')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



### Number of timesteps by `date_id` and `symbol_id`

In [ ]:
res = train_df.group_by("date_id", "symbol_id").agg(
    pl.col("time_id").count().alias("num_timesteps")
).sort(by=['date_id', 'symbol_id']).collect()

res

### Visualize the number of timesteps by `date_id` and `symbol_id`

In [ ]:
plt.figure(figsize=(12, 6))
for symbol in [1, 20]:
    symbol_data = res.filter(pl.col("symbol_id") == symbol).to_pandas()
    plt.plot(symbol_data["date_id"], symbol_data["num_timesteps"], label=f'Symbol {symbol}')

plt.title('Timesteps for Symbol ID 1 and 20')
plt.xlabel('Date ID')
plt.ylabel('Number of Timesteps')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## 1. NaN Analysis

In [ ]:
# get group 8 as our analysis target
sorted_df_symbol_id = train_df.filter(pl.col("symbol_id") == 8).sort(by=['date_id', 'time_id'])
sorted_df_symbol_id.collect()

2. Compute statistics for NaN's, **=> NaN's occur in the beginning of time series for**
   - **feature_21, feature_26, feature_27, feature_31**
   - **feature_00, feature_01, feature_02, feature_03, feature_04**

In [ ]:
# # 1. Count of NaNs for each row
feature_cols = [f"feature_{i:02d}" for i in range(79)]

sorted_df_symbol_id = sorted_df_symbol_id.with_columns(
    pl.fold(
        acc=pl.lit(0),  # Starting accumulator value
        function=lambda acc, col: acc + col.is_null().cast(pl.Int32),  # Increment for null values
        exprs=[pl.col(col) for col in feature_cols]  # List of columns to process
    ).alias('feature_nan_count')
)

sorted_df_symbol_id.collect()


In [22]:
# # 2. Top 10 columns with most NaNs
# column_nan_count = sorted_df_symbol_id.select(pl.col(feature_cols).null_count())
# print("Top 10 columns with most NaNs")
# column_nan_count.collect().to_pandas().sum().sort_values(ascending=False).head(10)

In [23]:
# # 3. Features without NaNs
# features_without_nan = [
#     col for col in feature_cols 
#     if sorted_df_symbol_id.select(pl.col(col).null_count().alias('nulls')).collect()['nulls'][0] == 0
# ]
# features_with_nans = [
#     col for col in feature_cols 
#     if sorted_df_symbol_id.select(pl.col(col).null_count().alias('nulls')).collect()['nulls'][0] > 0
# ]
# print("features_without_nan", features_without_nan)

In [24]:
# # 4. Calculate the percentage of NaNs for each feature
# feature_nan_percentages = (
#     sorted_df_symbol_id.select(
#         [
#             (pl.col(col).is_null() / sorted_df_symbol_id.collect().shape[0] * 100).alias(col)
#             for col in feature_cols
#         ]
#     ).collect()
#     .to_pandas()
#     .mean()
#     .sort_values(ascending=False)
# )

# # Display the top 10 features with the highest percentage of NaNs
# print("Percentage of NaNs for each feature:")
# print(feature_nan_percentages.head(10))


In [25]:
# # show the features with the highest percentage of NaNs
# features_with_most_nan = feature_nan_percentages.head(10).index
# result = sorted_df_symbol_id.select(features_with_most_nan)

# result.collect()

3. Visualize correlation matrix between features in which NaN occurs, **=> NaN occurs at same timestamps for**
   - **{feature_32-33, feature_58, feature_73-74}**
   - **{feature_39, feature_42, feature_50, feature_53}**
   - **{feature_40, feature_43}**
   - **{feature_41, feature_44, feature_52, feature_55}**
   - **{feature_45-46, feature_62-66}**
   - **{feature_51, feature_54}**
   - **{feature_75-76}**

In [26]:
# # Create a boolean DataFrame of NaN values
# nan_bool_df = sorted_df_symbol_id.select(features_with_nans).collect().to_pandas()

# # Calculate correlation matrix
# nan_corr_matrix = nan_bool_df.corr()

# # Visualize
# plt.figure(figsize=(12, 12)) 
# sns.heatmap(nan_corr_matrix, annot=True, cmap='coolwarm', fmt='.1f', vmin=-1, vmax=1, cbar=False)
# plt.title('Correlation Matrix of NaN Values')
# plt.tight_layout()
# plt.show()

## 2. Feature Analysis
1. Compute standard deviation for each feature on symbol_id=8, **=> feature_09, feature_10, feature_11 might depend on instrument since they have 0 stddev.** 

In [ ]:
feature_std = sorted_df_symbol_id.select(feature_cols).collect().to_pandas().std()
print("min", feature_std.min())
print("max", feature_std.max())
print(feature_std)

In [ ]:
zero_std_mask = feature_std == 0
feature_std[zero_std_mask]

1. Find maximum/minimum standard deviation for features on all `symbol_id`'s, then plot the features with symbol_id=8.

In [ ]:
# Calculate standard deviation
feature_std = train_df.select(feature_cols).collect().to_pandas().std()

# Drop specific columns
feature_std_filtered = feature_std.drop(["feature_09", "feature_10", "feature_11"])

# Identify features with max and min std
argmax_idx = feature_std_filtered.idxmax()
argmin_idx = feature_std_filtered.idxmin()

# Plot the values of these features
max_std_values = sorted_df_symbol_id.select(argmax_idx).collect().to_pandas()
min_std_values = sorted_df_symbol_id.select(argmin_idx).collect().to_pandas()

plt.plot(max_std_values, label=f'{argmax_idx} (max std)')
plt.plot(min_std_values, label=f'{argmin_idx} (min std)')

plt.xlabel("Index")
plt.ylabel("Value")
plt.title(f"Plot of {argmax_idx} (max std) and {argmin_idx} (min std)")

plt.legend()
plt.show()


In [ ]:
# Plot the feature with max std (argmax_idx) separately
plt.figure(figsize=(10, 5))
plt.plot(sorted_df_symbol_id.select(argmax_idx).collect().to_pandas(), label=f'{argmax_idx} (max std)', color='blue')
plt.xlabel("Index")
plt.ylabel("Value")
plt.title(f"Plot of {argmax_idx} (max std)")
plt.legend()
plt.show()

# Plot the feature with min std (argmin_idx) separately
plt.figure(figsize=(10, 5))
plt.plot(sorted_df_symbol_id.select(argmin_idx).collect().to_pandas(), label=f'{argmin_idx} (min std)', color='red')
plt.xlabel("Index")
plt.ylabel("Value")
plt.title(f"Plot of {argmin_idx} (min std)")
plt.legend()
plt.show()

1. Compute other feature statistics on `symbol_id`=8.

In [ ]:
feature_mean = sorted_df_symbol_id.select(feature_cols).collect().to_pandas().mean()
print(feature_mean) 
print("min", feature_mean.min())
print("max", feature_mean.max())
print(feature_mean[["feature_09","feature_10","feature_11"]])

In [ ]:
feature_max = sorted_df_symbol_id.select(feature_cols).collect().to_pandas().drop(columns=["feature_09", "feature_10", "feature_11"]).max()
print(feature_max) 
print("min", feature_max.min())
print("max", feature_max.max())

In [ ]:
feature_min = sorted_df_symbol_id.select(feature_cols).collect().to_pandas().drop(columns=["feature_09", "feature_10", "feature_11"]).min()
print(feature_min) 
print("min", feature_min.min())
print("max", feature_min.max())

In [ ]:
correlation_matrix = sorted_df_symbol_id.select(feature_cols).collect().to_pandas().corr()

plt.figure(figsize=(12, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".1f", 
            linewidths=0.5)

plt.title("Correlation heatmap of features 00 to 78, symbol_id=8")
plt.xlabel("Features")
plt.ylabel("Features")

plt.show()

In [ ]:
feature_list = ["feature_09", "feature_10", "feature_11"]
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharey=True)

for i, feature in enumerate(feature_list):
    # Convert the column to a Pandas Series
    pandas_series = train_df.select(feature).collect().to_pandas()[feature]
    
    # Count unique values
    unique_values = pandas_series.value_counts()
    print(f"Number of unique values for {feature}: {len(unique_values)}")
    
    # Create a bar plot
    sns.barplot(x=unique_values.index, y=unique_values.values, color='skyblue', edgecolor='black', ax=axes[i])
    axes[i].set_title(f"Unique values for {feature}")
    axes[i].set_xlabel("UniqueValue")
    axes[i].set_ylabel("Count")

plt.tight_layout()
plt.show()

## 3. How could *weight* be determined?
1. Compute correlation between `weight` and each feature (Pearson correlation measures the linear relationship between two variables, ranging from -1 (perfect negative correlation) to +1 (perfect positive correlation), with 0 indicating no linear relationship; Spearman correlation is a non-parametric measure of the strength and direction of the monotonic relationship between two variables) ignoring `nan`s **=> feature_21 and feature_31 are highly correlated to weight.**

2. TODO: check if dependent on `symbol_id`?

3. TODO: check trend w.r.t. `time_id`? (highly related)

In [ ]:
weight_feature_corrs = []
for feature in feature_cols:
    df_pandas = sorted_df_symbol_id.select('weight', feature).collect().to_pandas()
    weight_feature_corr = df_pandas['weight'].corr(df_pandas[feature], method='spearman')
    weight_feature_corrs.append((feature, weight_feature_corr))

highly_correlated = [feature for feature, corr in weight_feature_corrs if np.abs(corr) > 0.7]
moderately_correlated = [feature for feature, corr in weight_feature_corrs if 0.3 < np.abs(corr) <= 0.7]

print("Highly correlated features:", highly_correlated)
print("Moderately correlated features:", moderately_correlated)

In [ ]:
# Create a 1x2 grid of subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Scatter plot between 'weight' and 'feature_21' on the first subplot
axes[0].scatter(sorted_df_symbol_id.select("weight").collect().to_pandas(), sorted_df_symbol_id.select("feature_21").collect().to_pandas())
axes[0].set_xlabel("Weight")
axes[0].set_ylabel("Feature 21")
axes[0].set_title("Weight vs Feature 21")

# Scatter plot between 'weight' and 'feature_31' on the second subplot
axes[1].scatter(sorted_df_symbol_id.select("weight").collect().to_pandas(), sorted_df_symbol_id.select("feature_31").collect().to_pandas())
axes[1].set_xlabel("Weight")
axes[1].set_ylabel("Feature 31")
axes[1].set_title("Weight vs Feature 31")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))

# Plot for Feature 21
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(sorted_df_symbol_id.select("feature_21").collect().to_pandas(), label="Feature 21", color="blue")
plt.xlabel("Index")
plt.ylabel("Feature 21 Value")
plt.title("Plot of Feature 21")
plt.legend()
plt.grid(True)

# Plot for Feature 31
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(sorted_df_symbol_id.select("feature_31").collect().to_pandas(), label="Feature 31", color="green")
plt.xlabel("Index")
plt.ylabel("Feature 31 Value")
plt.title("Plot of Feature 31")
plt.legend()
plt.grid(True)

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

### Correlation between `symbol_id` and `weight`

In [ ]:
# calculate average weight for each symbol_id
avg_weight = train_df.group_by("symbol_id").agg(
    pl.col("weight").mean().alias("avg_weight")
).collect().to_pandas()

# plot average weight for each symbol_id
plt.figure(figsize=(8, 6))
sns.barplot(x=avg_weight["symbol_id"], y=avg_weight["avg_weight"])
plt.xlabel("Symbol ID")
plt.ylabel("Average Weight")
plt.xticks(rotation=90)
plt.title("Average Weight by Symbol ID")
plt.show()

### Correlation between `date_id`/`time_id` and `weight`

In [ ]:
# calculate average weight by date_id
avg_weight = train_df.group_by("date_id").agg(
    pl.col("weight").mean().alias("avg_weight")
).collect().to_pandas()

# plot average weight by date_id
plt.figure(figsize=(12, 6))
sns.lineplot(x=avg_weight["date_id"], y=avg_weight["avg_weight"])
plt.xlabel("Date ID")
plt.ylabel("Average Weight")
plt.title("Average Weight by Date ID")
plt.show()

In [ ]:
# calculate correlation between weight and date_id
correlation = train_df.select('weight', 'date_id').collect().to_pandas().corr(method='spearman')
print(correlation)

In [ ]:
# calculate average weight by time_id
avg_weight = train_df.group_by("time_id").agg(
    pl.col("weight").mean().alias("avg_weight")
).collect().to_pandas()

# plot average weight by time_id
plt.figure(figsize=(12, 6))
sns.lineplot(x=avg_weight["time_id"], y=avg_weight["avg_weight"])
plt.xlabel("Time ID")
plt.ylabel("Average Weight")
plt.title("Average Weight by Time ID")
plt.show()

In [ ]:
# calculate correlation between weight and time_id
correlation = train_df.select('weight', 'time_id').collect().to_pandas().corr(method='spearman')
print(correlation)

## 4. More on Responders

In [ ]:
responder_columns = [f"responder_{i}" for i in range(9)]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for i, column in enumerate(responder_columns):
    axes[i].plot(sorted_df_symbol_id.select(column).collect().to_pandas(), label=column)
    axes[i].set_xlabel("Row Index")
    axes[i].set_ylabel(f"Responder {i}")
    axes[i].set_title(f"Responder {i}")
    axes[i].legend()

plt.tight_layout()
plt.show()

1. Visualize Spearman correlation between responders.

In [ ]:
correlation_matrix = sorted_df_symbol_id.select(responder_columns).collect().to_pandas().corr(method='spearman')

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
            linewidths=0.5, cbar_kws={'label': 'Correlation Coefficient'})

plt.title("Spearman correlation between responders, symbol_id=8")
plt.xlabel("Responders")
plt.ylabel("Responders")

plt.show()

In [44]:
# too computationally expensive
# correlation_matrix = train_df.select(responder_columns).collect().to_pandas().corr(method='spearman')

# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
#             linewidths=0.5, cbar_kws={'label': 'Correlation Coefficient'})

# plt.title("Spearman correlation between responders, symbol_id=8")
# plt.xlabel("Responders")
# plt.ylabel("Responders")

# plt.show()

## 5. More on instrument independent analysis (the above analysis is on a single instrument that is randomly selected).